<a href="https://colab.research.google.com/github/vinbaskaran/AI_projects/blob/main/helpmate_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install all the required libraries

!pip install -U -q pdfplumber tiktoken openai chromaDB sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 779.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.7/767.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.7 MB/s et

In [3]:
import pdfplumber
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import openai
import chromadb

In [4]:
# Define path to the PDF file
pdf_file_path = Path("Principal-Sample-Life-Insurance-Policy.pdf")
print(f"PDF file path: {pdf_file_path}")
print(f"File exists: {pdf_file_path.exists()}")

PDF file path: Principal-Sample-Life-Insurance-Policy.pdf
File exists: True


In [5]:
# Open the PDF file
with pdfplumber.open(pdf_file_path) as pdf:

    # Get one of the pages from the PDF and examine it
    single_page = pdf.pages[9]

    # Extract text from the first page
    text = single_page.extract_text()

    # Extract tables from the first page
    tables = single_page.extract_tables()

    # Print the extracted text
    print(text)

T he legally recognized union of two eligible individuals of the same sex established according to
law.
Civil Union Partner
For two persons to establish a Civil Union in Rhode Island, it shall be necessary that they satisfy
all of the following criteria:
a. not be a party to another Civil Union or marriage in Rhode Island;
b. be of the same sex and therefore be excluded from the marriage laws of Rhode Island or
any other state;
c. be at least 18 years of age;
d. not be related to the other proposed party to the Civil Union.
NOTE: For the purposes of this Group Policy, the term "spouse" will include Civil Union
Partner, except as otherwise provided in this Group Policy.
Date of Issue
The date this Group Policy is placed in force: November 1, 2007.
Dependent
a. A Member's spouse, if that spouse:
(1) is legally married to the Member; and
(2) is not in the Armed Forces of any country; and
(3) is not insured under this Group Policy as a Member.
A Member's spouse will also include a Civil Un

In [6]:
# Function to extract text from a PDF file.
# 1. Declare a variable p to store the iteration of the loop that will help us store page numbers alongside the text
# 2. Declare an empty list 'full_text' to store all the text files
# 3. Use pdfplumber to open the pdf pages one by one
# 4. Find the tables and their locations in the page
# 5. Extract the text from the tables in the variable 'tables'
# 6. Extract the regular words by calling the function check_bboxes() and checking whether words are present in the table or not
# 7. Use the cluster_objects utility to cluster non-table and table words together so that they retain the same chronology as in the original PDF
# 8. Declare an empty list 'lines' to store the page text
# 9. If a text element in present in the cluster, append it to 'lines', else if a table element is present, append the table
# 10. Append the page number and all lines to full_text, and increment 'p'
# 11. When the function has iterated over all pages, return the 'full_text' list

def extract_text_from_pdf(pdf_path):
    p = 0
    full_text = []


    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_no = f"Page {p+1}"
            text = page.extract_text()

            tables = page.find_tables()
            table_bboxes = [i.bbox for i in tables]
            tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
            non_table_words = [word for word in page.extract_words() if not any(
                [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
            lines = []

            for cluster in pdfplumber.utils.cluster_objects(non_table_words + tables, itemgetter('top'), tolerance=5):

                if 'text' in cluster[0]:
                    try:
                        lines.append(' '.join([i['text'] for i in cluster]))
                    except KeyError:
                        pass

                elif 'table' in cluster[0]:
                    lines.append(json.dumps(cluster[0]['table']))


            full_text.append([page_no, " ".join(lines)])
            p +=1

    return full_text

In [7]:
full_text = extract_text_from_pdf(pdf_file_path)

In [8]:
extracted_text_df = pd.DataFrame(full_text, columns=['Page No.', 'Page_Text'])
extracted_text_df['Document Name'] = pdf_file_path.name

In [9]:
extracted_text_df

,Page No.,Page_Text,Document Name
0,Page 1,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,Principal-Sample-Life-Insurance-Policy.pdf
1,Page 2,This page left blank intentionally,Principal-Sample-Life-Insurance-Policy.pdf
2,Page 3,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,Principal-Sample-Life-Insurance-Policy.pdf
3,Page 4,This page left blank intentionally,Principal-Sample-Life-Insurance-Policy.pdf
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,Principal-Sample-Life-Insurance-Policy.pdf
...,...,...,...
59,Page 60,I f a Dependent who was insured dies during th...,Principal-Sample-Life-Insurance-Policy.pdf
60,Page 61,Section D - Claim Procedures Article 1 - Notic...,Principal-Sample-Life-Insurance-Policy.pdf
61,Page 62,A claimant may request an appeal of a claim de...,Principal-Sample-Life-Insurance-Policy.pdf
62,Page 63,This page left blank intentionally,Principal-Sample-Life-Insurance-Policy.pdf


In [10]:
# Check one of the extracted page texts to ensure that the text has been correctly read

extracted_text_df.Page_Text[2]

'POLICY RIDER GROUP INSURANCE POLICY NO: S655 COVERAGE: Life EMPLOYER: RHODE ISLAND JOHN DOE Effective on the later of the Date of Issue of this Group Policy or March 1, 2005, the following will apply to your Policy: From time to time The Principal may offer or provide certain employer groups who apply for coverage with The Principal a Financial Services Hotline and Grief Support Services or any other value added service for the employees of that employer group. In addition, The Principal may arrange for third party service providers (i.e., optometrists, health clubs), to provide discounted goods and services to those employer groups who apply for coverage with The Principal or who become insureds/enrollees of The Principal. While The Principal has arranged these goods, services and/or third party provider discounts, the third party service providers are liable to the applicants/insureds/enrollees for the provision of such goods and/or services. The Principal is not responsible for the

In [11]:
# Let's also check the length of all the texts as there might be some empty pages or pages with very few words that we can drop

extracted_text_df['Text_Length'] = extracted_text_df['Page_Text'].apply(lambda x: len(x.split(' ')))

In [12]:
extracted_text_df['Text_Length']

,Text_Length
0,30
1,5
2,230
3,5
4,110
...,...
59,285
60,418
61,322
62,5


In [13]:
# Retain only the rows with a text length of at least 10

insurance_pdfs_data = extracted_text_df.loc[extracted_text_df['Text_Length'] >= 10]
insurance_pdfs_data

,Page No.,Page_Text,Document Name,Text_Length
0,Page 1,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,Principal-Sample-Life-Insurance-Policy.pdf,30
2,Page 3,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,Principal-Sample-Life-Insurance-Policy.pdf,230
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,Principal-Sample-Life-Insurance-Policy.pdf,110
5,Page 6,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,Principal-Sample-Life-Insurance-Policy.pdf,153
6,Page 7,Section A – Eligibility Member Life Insurance ...,Principal-Sample-Life-Insurance-Policy.pdf,176
7,Page 8,Section A - Member Life Insurance Schedule of ...,Principal-Sample-Life-Insurance-Policy.pdf,171
8,Page 9,P ART I - DEFINITIONS When used in this Group ...,Principal-Sample-Life-Insurance-Policy.pdf,387
9,Page 10,T he legally recognized union of two eligible ...,Principal-Sample-Life-Insurance-Policy.pdf,251
10,Page 11,(2) has been placed with the Member or spouse ...,Principal-Sample-Life-Insurance-Policy.pdf,299
11,Page 12,An institution that is licensed as a Hospital ...,Principal-Sample-Life-Insurance-Policy.pdf,352


In [14]:
# Enhanced metadata for insurance_pdfs_data
import datetime
import re

# Create a copy to add metadata
enhanced_data = insurance_pdfs_data.copy()


# 1. Page Number (numeric for sorting)
enhanced_data['Page_Number'] = enhanced_data['Page No.'].str.extract(r'(\d+)').astype(int)

# 2. Text Statistics
enhanced_data['Word_Count'] = enhanced_data['Page_Text'].apply(lambda x: len(x.split()))
enhanced_data['Character_Count'] = enhanced_data['Page_Text'].apply(len)
enhanced_data['Sentence_Count'] = enhanced_data['Page_Text'].apply(lambda x: len(re.split(r'[.!?]+', x)))

# 3. Content Classification (basic heuristics)
def classify_content(text):
    text_lower = text.lower()
    if any(word in text_lower for word in ['table of contents', 'contents']):
        return 'Table of Contents'
    elif any(word in text_lower for word in ['premium', 'benefit', 'coverage']):
        return 'Policy Details'
    elif any(word in text_lower for word in ['definition', 'definitions']):
        return 'Definitions'
    elif any(word in text_lower for word in ['rider', 'endorsement']):
        return 'Rider/Endorsement'
    elif any(word in text_lower for word in ['claim', 'claims']):
        return 'Claims Information'
    else:
        return 'General Content'

enhanced_data['Content_Category'] = enhanced_data['Page_Text'].apply(classify_content)

# 4. Text Quality Indicators
enhanced_data['Has_Tables'] = enhanced_data['Page_Text'].apply(lambda x: '[' in x and ']' in x)
enhanced_data['Text_Density'] = enhanced_data['Character_Count'] / (enhanced_data['Character_Count'].max() + 1)

print("Enhanced metadata columns added:")
print(enhanced_data.columns.tolist())
print(f"\nNew DataFrame shape: {enhanced_data.shape}")

Enhanced metadata columns added:
['Page No.', 'Page_Text', 'Document Name', 'Text_Length', 'Page_Number', 'Word_Count', 'Character_Count', 'Sentence_Count', 'Content_Category', 'Has_Tables', 'Text_Density']

New DataFrame shape: (60, 11)


In [15]:
# Display sample of enhanced data
print("Sample of enhanced insurance_pdfs_data with metadata:")
print(enhanced_data[['Page No.', 'Content_Category', 'Word_Count', 'Has_Tables', 'Text_Density']].head(10))

print("\nContent Category Distribution:")
print(enhanced_data['Content_Category'].value_counts())

print("\nPages with Tables:")
print(f"Total pages with tables: {enhanced_data['Has_Tables'].sum()}")

print("\nText Statistics Summary:")
print(enhanced_data[['Word_Count', 'Character_Count', 'Sentence_Count']].describe())

Sample of enhanced insurance_pdfs_data with metadata:
   Page No.   Content_Category  Word_Count  Has_Tables  Text_Density
0    Page 1    General Content          30       False      0.069578
2    Page 3     Policy Details         230       False      0.543301
4    Page 5     Policy Details         110       False      0.262398
5    Page 6  Table of Contents         153       False      0.378608
6    Page 7  Table of Contents         176       False      0.429312
7    Page 8  Table of Contents         171       False      0.398594
8    Page 9        Definitions         387       False      0.824574
9   Page 10        Definitions         251       False      0.510733
10  Page 11     Policy Details         299       False      0.655440
11  Page 12        Definitions         352       False      0.771281

Content Category Distribution:
Content_Category
Policy Details       49
Definitions           5
General Content       3
Table of Contents     3
Name: count, dtype: int64

Pages with Tabl

In [25]:
# Combine all metadata columns into a single dictionary column for ChromaDB

# Define which columns are metadata (exclude the main content columns)
content_columns = ['Page No.', 'Page_Text', 'Document Name']
metadata_columns = [col for col in enhanced_data.columns if col not in content_columns]

print("Metadata columns to combine:")
print(metadata_columns)

# Function to create metadata dictionary for each row
def create_metadata_dict(row):
    metadata_dict = {}
    for col in metadata_columns:
        value = row[col]
        # Convert timestamps to string for JSON compatibility
        if isinstance(value, pd.Timestamp):
            metadata_dict[col] = value.isoformat()
        # Convert numpy/pandas types to native Python types
        elif hasattr(value, 'item'):
            metadata_dict[col] = value.item()
        else:
            metadata_dict[col] = value
    return metadata_dict

# Create the combined metadata column
enhanced_data['metadata'] = enhanced_data.apply(create_metadata_dict, axis=1)

# Display sample of the new metadata column
print(f"\nSample metadata dictionary:")
print(enhanced_data['metadata'].iloc[0])

print(f"\nDataFrame now has {len(enhanced_data.columns)} columns")
print(f"New column 'metadata' contains dictionaries with {len(metadata_columns)} metadata fields")

Metadata columns to combine:
['Text_Length', 'Page_Number', 'Word_Count', 'Character_Count', 'Sentence_Count', 'Content_Category', 'Has_Tables', 'Text_Density']

Sample metadata dictionary:
{'Text_Length': 30, 'Page_Number': 1, 'Word_Count': 30, 'Character_Count': 188, 'Sentence_Count': 1, 'Content_Category': 'General Content', 'Has_Tables': False, 'Text_Density': 0.0695780903034789}

DataFrame now has 12 columns
New column 'metadata' contains dictionaries with 8 metadata fields


In [32]:
# Set the API key
with open("OpenAI_API_Key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())
#import os
#from google.colab import userdata
#os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [33]:
# Import the OpenAI Embedding Function into chroma

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [34]:
# Define the path where chroma collections will be stored

chroma_data_path = 'ChromaDB_Data'

In [35]:
import chromadb

In [36]:
# Call PersistentClient()

client = chromadb.PersistentClient()

In [37]:
# Set up the embedding function using the OpenAI embedding model

model = "text-embedding-ada-002"
embedding_function = OpenAIEmbeddingFunction(api_key=openai.api_key, model_name=model)

In [38]:
# Initialise a collection in chroma and pass the embedding_function to it so that it used OpenAI embeddings to embed the documents

insurance_collection = client.get_or_create_collection(name='RAG_on_Insurance', embedding_function=embedding_function)

In [39]:
# Convert the page text and metadata from your dataframe to lists to be able to pass it to chroma

documents_list = enhanced_data["Page_Text"].tolist()
metadata_list = enhanced_data['metadata'].tolist()

In [40]:
# Add the documents and metadata to the collection alongwith generic integer IDs. You can also feed the metadata information as IDs by combining the policy name and page no.

insurance_collection.add(
    documents= documents_list,
    ids = [str(i) for i in range(0, len(documents_list))],
    metadatas = metadata_list
)

In [41]:
# Let's take a look at the first few entries in the collection

insurance_collection.get(
    ids = ['0','1','2'],
    include = ['embeddings', 'documents', 'metadatas']
)

{'ids': ['0', '1', '2'],
 'embeddings': array([[-2.24228799e-02,  1.87183432e-02, -2.72361692e-02, ...,
         -3.69149223e-02,  2.83710100e-03, -1.30930578e-03],
        [-1.32036684e-02,  8.89394712e-03, -4.63569537e-03, ...,
         -1.57016590e-02, -4.11756810e-05,  7.26064527e-03],
        [-1.21371727e-02,  1.40738143e-02, -3.41565232e-03, ...,
         -2.85190400e-02, -9.45770927e-03,  1.02336928e-02]]),
 'documents': ['DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/01/2014 711 HIGH STREET GEORGE RI 02903 GROUP POLICY FOR: RHODE ISLAND JOHN DOE ALL MEMBERS Group Member Life Insurance Print Date: 07/16/2014',
  'POLICY RIDER GROUP INSURANCE POLICY NO: S655 COVERAGE: Life EMPLOYER: RHODE ISLAND JOHN DOE Effective on the later of the Date of Issue of this Group Policy or March 1, 2005, the following will apply to your Policy: From time to time The Principal may offer or provide certain employer groups who apply for coverage with The Principal a Financial Services Hotline and Gri

In [42]:
cache_collection = client.get_or_create_collection(name='Insurance_Cache', embedding_function=embedding_function)

In [43]:
cache_collection.peek()

{'ids': [],
 'embeddings': array([], dtype=float64),
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents', 'embeddings'],
 'data': None,
 'metadatas': []}

In [44]:
# Read the user query

query = input()

What are the death benefits


In [45]:
# Searh the Cache collection first
# Query the collection against the user query and return the top 20 results

cache_results = cache_collection.query(
    query_texts=query,
    n_results=1
)

In [46]:
cache_results

{'ids': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[]],
 'distances': [[]]}

In [47]:
results = insurance_collection.query(
query_texts=query,
n_results=10
)
# results.items()

In [49]:
# Implementing Cache in Semantic Search

# Set a threshold for cache search
threshold = 0.2

ids = []
documents = []
distances = []
metadatas = []
results_df = pd.DataFrame()


# If the distance is greater than the threshold, then return the results from the main collection.

if cache_results['distances'][0] == [] or cache_results['distances'][0][0] > threshold:
      # Query the collection against the user query and return the top 10 results
      results = insurance_collection.query(
      query_texts=query,
      n_results=10
      )

      # Store the query in cache_collection as document w.r.t to ChromaDB so that it can be embedded and searched against later
      # Store retrieved text, ids, distances and metadatas in cache_collection as metadatas, so that they can be fetched easily if a query indeed matches to a query in cache
      Keys = []
      Values = []

      for key, val_list in results.items():
          if val_list is None or not val_list:
              continue
          for i, val in enumerate(val_list[0]):
              Keys.append(str(key) + str(i))
              # Ensure value is string for dictionary key-value
              Values.append(str(val))


      cache_collection.add(
          documents= [query],
          ids = [query],  # Or if you want to assign integers as IDs 0,1,2,.., then you can use "len(cache_results['documents'])" as will return the no. of queries currently in the cache and assign the next digit to the new query."
          metadatas = dict(zip(Keys, Values))
      )

      print("Not found in cache. Found in main collection.")

      result_dict = {'Metadatas': results['metadatas'][0], 'Documents': results['documents'][0], 'Distances': results['distances'][0], "IDs":results["ids"][0]}
      results_df = pd.DataFrame.from_dict(result_dict)
      results_df


# If the distance is, however, less than the threshold, you can return the results from cache

elif cache_results['distances'][0][0] <= threshold:
      cache_result_dict = cache_results['metadatas'][0][0]

      # Loop through each inner list and then through the dictionary
      for key, value in cache_result_dict.items():
          if 'ids' in key:
              ids.append(value)
          elif 'documents' in key:
              documents.append(value)
          elif 'distances' in key:
              distances.append(value)
          elif 'metadatas' in key:
              metadatas.append(value)

      print("Found in cache!")

      # Create a DataFrame
      results_df = pd.DataFrame({
        'IDs': ids,
        'Documents': documents,
        'Distances': distances,
        'Metadatas': metadatas
      })

Not found in cache. Found in main collection.


In [50]:
results_df

,Metadatas,Documents,Distances,IDs
0,"{'Sentence_Count': 21, 'Content_Category': 'Po...","M ember's death, the Death Benefits Payable ma...",0.319182,44
1,"{'Has_Tables': False, 'Content_Category': 'Tab...",Section A - Member Life Insurance Schedule of ...,0.335242,5
2,"{'Character_Count': 1670, 'Page_Number': 60, '...",I f a Dependent who was insured dies during th...,0.352003,57
3,"{'Content_Category': 'Policy Details', 'Charac...",% of Scheduled Covered Loss Benefit Loss of Sp...,0.352851,54
4,"{'Has_Tables': False, 'Text_Length': 368, 'Cha...","f . claim requirements listed in PART IV, Sect...",0.353494,51
5,"{'Sentence_Count': 13, 'Text_Length': 429, 'Co...",(1) marriage or establishment of a Civil Union...,0.360397,29
6,"{'Character_Count': 1632, 'Page_Number': 51, '...",Coverage During Disability will cease on the e...,0.370392,48
7,"{'Text_Length': 420, 'Content_Category': 'Poli...",c . If a beneficiary dies at the same time or ...,0.372374,45
8,"{'Text_Density': 0.46039970392302, 'Sentence_C...",(1) only one Accelerated Benefit payment will ...,0.378658,49
9,"{'Word_Count': 179, 'Sentence_Count': 10, 'Tex...",(1) If termination is as described in b. (1) a...,0.385215,42


In [51]:
# Import the CrossEncoder library from sentence_transformers

from sentence_transformers import CrossEncoder, util

In [52]:
# Initialise the cross encoder model

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [53]:
# Test the cross encoder model

scores = cross_encoder.predict([['Does the insurance cover diabetic patients?', 'The insurance policy covers some pre-existing conditions including diabetes, heart diseases, etc. The policy does not howev'],
                                ['Does the insurance cover diabetic patients?', 'The premium rates for various age groups are given as follows. Age group (<18 years): Premium rate']])

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [54]:
scores

array([  3.8467617, -11.252879 ], dtype=float32)

In [55]:
# Input (query, response) pairs for each of the top 20 responses received from the semantic search to the cross encoder
# Generate the cross_encoder scores for these pairs

cross_inputs = [[query, response] for response in results_df['Documents']]
cross_rerank_scores = cross_encoder.predict(cross_inputs)

In [56]:
cross_rerank_scores

array([ 0.96486413,  0.43309438, -0.6457482 , -0.5766581 , -3.509951  ,
       -2.76471   , -8.083011  , -3.0920267 , -2.6270356 , -7.6952405 ],
      dtype=float32)

In [57]:
# Store the rerank_scores in results_df

results_df['Reranked_scores'] = cross_rerank_scores

In [58]:
results_df

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,"{'Sentence_Count': 21, 'Content_Category': 'Po...","M ember's death, the Death Benefits Payable ma...",0.319182,44,0.964864
1,"{'Has_Tables': False, 'Content_Category': 'Tab...",Section A - Member Life Insurance Schedule of ...,0.335242,5,0.433094
2,"{'Character_Count': 1670, 'Page_Number': 60, '...",I f a Dependent who was insured dies during th...,0.352003,57,-0.645748
3,"{'Content_Category': 'Policy Details', 'Charac...",% of Scheduled Covered Loss Benefit Loss of Sp...,0.352851,54,-0.576658
4,"{'Has_Tables': False, 'Text_Length': 368, 'Cha...","f . claim requirements listed in PART IV, Sect...",0.353494,51,-3.509951
5,"{'Sentence_Count': 13, 'Text_Length': 429, 'Co...",(1) marriage or establishment of a Civil Union...,0.360397,29,-2.764710
6,"{'Character_Count': 1632, 'Page_Number': 51, '...",Coverage During Disability will cease on the e...,0.370392,48,-8.083011
7,"{'Text_Length': 420, 'Content_Category': 'Poli...",c . If a beneficiary dies at the same time or ...,0.372374,45,-3.092027
8,"{'Text_Density': 0.46039970392302, 'Sentence_C...",(1) only one Accelerated Benefit payment will ...,0.378658,49,-2.627036
9,"{'Word_Count': 179, 'Sentence_Count': 10, 'Tex...",(1) If termination is as described in b. (1) a...,0.385215,42,-7.695240


In [59]:
# Return the top 3 results from semantic search

top_3_semantic = results_df.sort_values(by='Distances')
top_3_semantic[:3]

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,"{'Sentence_Count': 21, 'Content_Category': 'Po...","M ember's death, the Death Benefits Payable ma...",0.319182,44,0.964864
1,"{'Has_Tables': False, 'Content_Category': 'Tab...",Section A - Member Life Insurance Schedule of ...,0.335242,5,0.433094
2,"{'Character_Count': 1670, 'Page_Number': 60, '...",I f a Dependent who was insured dies during th...,0.352003,57,-0.645748


In [60]:
# Return the top 3 results after reranking

top_3_rerank = results_df.sort_values(by='Reranked_scores', ascending=False)
top_3_rerank[:3]

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,"{'Sentence_Count': 21, 'Content_Category': 'Po...","M ember's death, the Death Benefits Payable ma...",0.319182,44,0.964864
1,"{'Has_Tables': False, 'Content_Category': 'Tab...",Section A - Member Life Insurance Schedule of ...,0.335242,5,0.433094
3,"{'Content_Category': 'Policy Details', 'Charac...",% of Scheduled Covered Loss Benefit Loss of Sp...,0.352851,54,-0.576658


In [61]:
top_3_RAG = top_3_rerank[["Documents", "Metadatas"]][:3]

In [62]:
top_3_RAG

,Documents,Metadatas
0,"M ember's death, the Death Benefits Payable ma...","{'Sentence_Count': 21, 'Content_Category': 'Po..."
1,Section A - Member Life Insurance Schedule of ...,"{'Has_Tables': False, 'Content_Category': 'Tab..."
3,% of Scheduled Covered Loss Benefit Loss of Sp...,"{'Content_Category': 'Policy Details', 'Charac..."


In [63]:
# Define the function to generate the response. Provide a comprehensive prompt that passes the user query and the top 3 results to the model

def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
                {"role": "system", "content":  "You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents."},
                {"role": "user", "content": f"""You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents.
                                                You have a question asked by the user in '{query}' and you have some search results from a corpus of insurance documents in the dataframe '{top_3_RAG}'. These search results are essentially one page of an insurance document that may be relevant to the user query.

                                                The column 'documents' inside this dataframe contains the actual text from the policy document and the column 'metadata' contains the policy name and source page. The text inside the document may also contain tables in the format of a list of lists where each of the nested lists indicates a row.

                                                Use the documents in '{top_3_RAG}' to answer the query '{query}'. Frame an informative answer and also, use the dataframe to return the relevant policy names and page numbers as citations.

                                                Follow the guidelines below when performing the task.
                                                1. Try to provide relevant/accurate numbers if available.
                                                2. You don’t have to necessarily use all the information in the dataframe. Only choose information that is relevant.
                                                3. If the document text has tables with relevant information, please reformat the table and return the final information in a tabular in format.
                                                3. Use the Metadatas columns in the dataframe to retrieve and cite the policy name(s) and page numbers(s) as citation.
                                                4. If you can't provide the complete answer, please also provide any information that will help the user to search specific sections in the relevant cited documents.
                                                5. You are a customer facing assistant, so do not provide any information on internal workings, just answer the query directly.

                                                The generated response should answer the query directly addressing the user and avoiding additional information. If you think that the query is not relevant to the document, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text along with the citation. Provide your complete response first with all information, and then provide the citations.
                                                """},
              ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return response.choices[0].message.content.split('\n')

In [64]:
# Generate the response

response = generate_response(query, top_3_RAG)

In [65]:
# Print the response

print("\n".join(response))

The death benefits under the insurance policy include a payout that is payable upon the death of the member. The exact amount of the death benefit can vary based on the specific policy terms and conditions. Additional details such as the specific payout amount or any additional benefits would typically be outlined in the policy document.

**Relevant Policy Information:**
- **Policy Name:** Member Life Insurance Policy
- **Page Number:** Section A

In the provided documents, the death benefits are described, and the specific details can be found in the Member Life Insurance Schedule of Benefits under Section A.

Please refer to the Member Life Insurance Schedule of Benefits under Section A in the documents for detailed information on the death benefits and any additional specifics related to the payout upon the member's death.
